Imports

In [2]:
import datetime
import pandas as pd
from pandas import DataFrame
from openpyxl import load_workbook,worksheet,Workbook

Constants

In [3]:
MYDIR= r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files"

PNs to be Removed

In [4]:
list_of_pns = [
    96699290, 97775274, 96699299, 97775277, 96778078,
    97780992, 96699305, 96769184, 97778033, 96769190,
    96769205, 97778039, 96769256, 96896891, 96769259,
    96769271, 97780970, 96769280, 97780973
]

str_list = [str(numstr) for numstr in list_of_pns]

Justification Note

In [5]:
def add_removal_note():
    timeStamp = datetime.datetime.now().strftime("%m-%d-%Y")
    reason = input("Reason for removal: ")
    authority = input("Who authorized/requested this change? ")
    return timeStamp + " " + reason + " per " + authority


Geeting the First Row of the PSD

In [6]:
wb=load_workbook(r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files\Lbom-ES.xlsx',read_only=True)

In [7]:
def get_start_row(wb:Workbook,sheet_name:str,min_row:int=None,max_row:int=None,min_col:int=None,max_col:int=None):
    try:
        if not sheet_name in wb.sheetnames:
            raise Exception
        ws=wb[sheet_name]
    except:
        print('Sheet does not exist')
    min_col = min_col or 1
    min_row = min_row or 1
    max_col = max_col or ws.max_column
    max_row = max_row or ws.max_row
    for row in ws.iter_rows(min_row,max_row,min_col,max_col):
        if row[0].value=='[START]':
            return row[0].row-1

psd_startrow=get_start_row(wb,'Impeller',min_row=1,max_col=1)
psd_startcol=0
        
    

Grouping Function

In [8]:
def group_then_separate_by(psd_data:DataFrame,list_of_cols: list, pn_col: str)->tuple[DataFrame,DataFrame]:
    """list of cols are grouping categories. pn_col is the column that contains pns to find matches on."""
    groups = psd_data.groupby(list_of_cols) # Had to play around with this to get the right groupings

    df_list_to_remove = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later
    df_list_to_keep   = [] # will hold list of dataframes to remain in PSD. Will concatenate to 1 dataframe later

    # Iterates through each sub-group, checking if there is a pn that meets criteria for removal
    for _, frame in groups:
        if any(frame[pn_col].isin(str_list)):   # Finding matching partnumbers to remove
            df_list_to_remove.append(frame)     # Need to add this sub-group to a "removed dataframe"
        else:
            df_list_to_keep.append(frame)       # Need to retain this sub-group, add to a "keep dataframe"

    # Concatenating list of dfs to single dfs.
    removals = pd.concat(df_list_to_remove)
    keep = pd.concat(df_list_to_keep)
        
    return removals, keep

Iterate Through Directory

In [28]:
import os
import numpy as np
from typing import Callable,Any,ParamSpec,TypeVar,Concatenate
from _utils.file_ops import read_files_in_dir
from _utils.Dataframe_tools import get_df

P = ParamSpec('P')



files=read_files_in_dir(MYDIR)
sheetname='Impeller'


# for file in files:
#     removal_note=add_removal_note()
#     data,fname=file
#     psd_data=pd.read_excel(data,sheet_name=sheetname,header=psd_startrow-1,dtype={'BOM':str})
#     original_psd_bottom_row = len(psd_data) + psd_startrow
#     print("Opening file for updates: {}".format(os.path.basename(fname)))
#     end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
#     psd_data = psd_data.iloc[:end_row]
#     group_by_columns = ["Model", "Price ID"]
#     removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
#     removals.loc[removals["Full Data"] == "[START]", "Full Data"] = ""
#     new_row = pd.DataFrame({'ID': removal_note}, index =[0])
#     removals = pd.concat([new_row, removals[:]]).reset_index()
#     column_list = keep.columns
#     removals = removals[column_list]
#     removals.sort_values(by=['ID'], inplace=True)
#     keep.loc[0,'Full Data'] = np.nan
#     keep.sort_values(by=['ID'], inplace=True)
#     keep.reset_index(drop=True, inplace=True)
#     keep.loc[0,'Full Data'] = "[START]"

#Assuming only one sheet for right now
def before_change_data(dir,sheet_name:list[list[str]],is_same_note=True):
    files=read_files_in_dir(dir)
    for i,file in files:
        header=[]
        note=add_removal_note()
        for sheet in sheet_name[i]:
            header.append(get_start_row(load_workbook(file[0],read_only=True),sheet_name=sheet,min_row=1,max_col=1))
            if not is_same_note:
                note=add_removal_note()
            

    # if is_same_note:
    #     note=add_removal_note()
    #     return ([file[0],file[1],note] for file in files) 
    # else:
    #     return ([file[0],file[1],add_removal_note()] for file in files)



# a=list(before_change_data(MYDIR))

"""Challenges are """

def change_data(file:tuple[bytes,str,str],psd_data_func:Callable[Concatenate[P],DataFrame],psd_data_args:tuple[list,dict]):
    removal_note=add_removal_note()
    data,fname=file
    psd_data=psd_data_func(data,*psd_data_args[0],**psd_data_args[1]) #will have to be called as a list or all have to be the same
    original_psd_bottom_row = len(psd_data) + psd_startrow
    print("Opening file for updates: {}".format(os.path.basename(fname)))
    end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
    psd_data = psd_data.iloc[:end_row]
    group_by_columns = ["Model", "Price ID"]
    removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
    removals.loc[removals["Full Data"] == "[START]", "Full Data"] = np.nan
    new_row = pd.DataFrame({'ID': removal_note}, index =[0])
    removals = pd.concat([new_row, removals[:]]).reset_index()
    column_list = keep.columns
    removals = removals[column_list]
    removals.sort_values(by=['ID'], inplace=True)
    keep.loc[0,'Full Data'] = np.nan
    keep.sort_values(by=['ID'], inplace=True)
    keep.reset_index(drop=True, inplace=True)
    keep.loc[0,'Full Data'] = "[START]"
    return keep,removals,fname



Adding in Multi-Processing

Can't be done in this notebook

In [2]:
from time import sleep
from random import random
from multiprocessing import Process
from multiprocessing import Event
 
# target task function
def task(event, number):
    # wait for the event to be set
    print(f'Process {number} waiting...', flush=True)
    event.wait()
    # begin processing
    value = random()
    sleep(value)
    print(f'Process {number} got {value}', flush=True)
 
# entry point
if __name__ == '__main__':
    # create a shared event object
    event = Event()
    # create a suite of processes
    processes = [Process(target=task, args=(event, i)) for i in range(5)]
    # start all processes
    for process in processes:
        process.start()
    # block for a moment
    print('Main process blocking...')
    sleep(2)
    # trigger all child processes
    event.set()
    # wait for all child processes to terminate
    for process in processes:
        process.join()

Main process blocking...
